<a href="https://colab.research.google.com/github/parvvaresh/NLP-text-summarizer-from-scratch/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing libraries
from google.colab import output
import nltk
import pandas as pd
import spacy
import string
import numpy as np
import re

# downloading required files (download if needed)
nltk.download('stopwords')
nltk.download('punkt')
!pip install wikipedia
import wikipedia
output.clear()

In [86]:
#from Wikipedia 
mahsa_amini = wikipedia.page("Mahsa amini")
mahsa_amini = mahsa_amini.content

#see it
mahsa_amini[: 1300]

"On 16 September 2022, the 22-year-old Iranian woman Mahsa Amini, also known as Jina Amini, died in a hospital in Tehran, Iran, under suspicious circumstances. The Guidance Patrol, the religious morality police of Iran's government, arrested Amini for allegedly not wearing the hijab in accordance with government standards. The Law Enforcement Command of the Islamic Republic of Iran stated that she had a heart attack at a police station, collapsed, and fell into a coma before being transferred to a hospital. However, eyewitnesses, including women who were detained with Amini, reported that she was severely beaten and that she died as a result of police brutality, which was denied by the Iranian authorities. The assertions of police brutality, in addition to leaked medical scans, led some observers to believe Amini had a cerebral hemorrhage or stroke due to head injuries received after her arrest.Amini's death resulted in a series of protests described by CNN as more widespread than the 

In [ ]:
# removing waste elements
mahsa_amini = re.sub('\[[a-zA-z0-9\s]+\]', '', mahsa_amini) # removing character like [34 text] etc
mahsa_amini = re.sub('=+\s+\d?-?[a-zA-z0-9\:–\s-]+\s+=+', '', mahsa_amini) # removing char

exclist = string.punctuation
# remove punctuations and digits from oldtext
table_ = str.maketrans('', '', exclist)
newtext = mahsa_amini.translate(table_).lower()

In [ ]:


def freq(text, stopword):
  word_count = {}
  txt_tokenize = nltk.tokenize.word_tokenize(text)

  txt_with_out_stpword = [word for word in txt_tokenize if word not in stopword]

  for word  in txt_with_out_stpword:
    if word in word_count:
      word_count[word] += 1
    else:
      word_count[word] = 1
  return word_count

stpWrd = nltk.corpus.stopwords.words('english')


word_count = freq(newtext, stpWrd)


word_count_table = pd.DataFrame({
    "key" : list(word_count.keys()),
    "value" : list(word_count.values())

})

word_count_table = word_count_table.sort_values(by = "value", ascending = False)


word_count_table


,key,value
7,amini,47
71,death,40
316,aminis,35
4,iranian,34
74,protests,29
...,...,...
540,nothing,1
546,wrapping,1
547,internal,1
548,injuriesaccording,1


In [80]:
sent_token = nltk.tokenize.sent_tokenize(mahsa_amini)


def sent_weight(Sentences, word_count):
  sent_rank = {}

  for Sentence in Sentences:
    for word, freq in word_count.items():
      if word in Sentence.lower():
        if Sentence in sent_rank:
          sent_rank[Sentence] += freq

        else:
          sent_rank[Sentence] = freq
      else:
        pass
  return sent_rank


rank_Sentence = sent_weight(sent_token, word_count)


In [ ]:
avg = sum(list(rank_Sentence.values())) / len(rank_Sentence)

print(f"Are all sentences : {len(rank_Sentence)} \nAnd the average point : {avg}\nTotal points : {sum(list(rank_Sentence.values()))}")

Are all sentences : 146 
And the average point : 129.21917808219177
Total points : 18866


In [78]:
summary  = ""

for Sentence in sent_token:
  if  (Sentence in  rank_Sentence) and (rank_Sentence[Sentence] > (1.15 * avg)):
    summary += (" " + Sentence)

In [87]:
summary[ : 1300]

" On 16 September 2022, the 22-year-old Iranian woman Mahsa Amini, also known as Jina Amini, died in a hospital in Tehran, Iran, under suspicious circumstances. The Guidance Patrol, the religious morality police of Iran's government, arrested Amini for allegedly not wearing the hijab in accordance with government standards. However, eyewitnesses, including women who were detained with Amini, reported that she was severely beaten and that she died as a result of police brutality, which was denied by the Iranian authorities. The assertions of police brutality, in addition to leaked medical scans, led some observers to believe Amini had a cerebral hemorrhage or stroke due to head injuries received after her arrest.Amini's death resulted in a series of protests described by CNN as more widespread than the protests in 2009, 2017, and 2019, and by The New York Times as the largest Iranian protests since at least 2009. Iran Human Rights reported that by December 2022 at least 476 people had b